In [5]:
import sqlite3
import pandas as pd

# File paths
source_db = 'fifa_world_cup.db'
target_db = 'shoot_out_advantage.db'

# Connect to source DB
conn_src = sqlite3.connect(source_db)

# Read selected columns
query = """
SELECT shoot_out, neutral_location, home_team_result
FROM fifa_world_cup
"""
df = pd.read_sql_query(query, conn_src)
conn_src.close()

# Track unexpected values in shoot_out
unexpected_values = set()

# Function to convert boolean-like values to 0/1
def to_bool_numeric(x):
    truthy = {'true', '1', 'yes', 'y', 't'}
    falsy = {'false', '0', 'no', 'n', 'f'}

    x_str = str(x).strip().lower()
    if x_str in truthy:
        return 1
    elif x_str in falsy:
        return 0
    else:
        unexpected_values.add(x_str)
        return 0  # Or use None if you'd prefer to keep unknowns as NaN

# Convert only the 'shoot_out' column
df['shoot_out'] = df['shoot_out'].map(to_bool_numeric)

# Show any unexpected values
if unexpected_values:
    print("⚠️ Unexpected 'shoot_out' values defaulted to 0:", unexpected_values)

# Save to new DB
conn_target = sqlite3.connect(target_db)
df.to_sql('fifa_world_cup', conn_target, index=False, if_exists='replace')
conn_target.close()

print("✅ New database 'shoot_out_advantage.db' created with 'shoot_out' converted to 0/1.")


✅ New database 'shoot_out_advantage.db' created with 'shoot_out' converted to 0/1.
